In [1]:
# For Google Colab
# Install folium package.
%pip install folium

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pickle

# 載入 graph.pkl（請確認跟 main.ipynb 在同一資料夾）
with open("graph.pkl", "rb") as f:
    graph = pickle.load(f)


In [3]:
# # Part 1~4 and 6
# # You can wirte in new .py files and import it.
# # ex: from astar import astar

# from bfs import bfs
# from dfs_stack import dfs
# #from dfs_recursive import dfs
# from ucs import ucs
# from astar import astar

In [4]:
import sys
sys.setrecursionlimit(10000)

In [5]:
import pandas as pd
import folium

df = pd.read_csv("taipei_attractions.csv")

fmap = folium.Map(location=(25.0330, 121.5654), zoom_start=13)

for _, row in df.iterrows():
    folium.Marker(
        location=(row["latitude"], row["longitude"]),
        popup=row["name"],
        icon=folium.Icon(color='green', icon='info-sign')
    ).add_to(fmap)

fmap


In [8]:
from ucs import load_graph, ucs
import folium
import pickle

# 讀圖
graph = load_graph("edges.csv")

# 起點與終點（這組有路徑）
start = 7728000683
end = 6331191918

# 執行 UCS
ucs_path, ucs_dist, ucs_visited = ucs(graph, start, end)
print(f'The number of nodes in the path found by UCS: {len(ucs_path)}')
print(f'Total distance of path found by UCS: {ucs_dist} m')
print(f'The number of visited nodes in UCS: {ucs_visited}\n')
# 地圖中心
fmap = folium.Map(location=[25.038, 121.564], zoom_start=15)

# 這是幫你畫路徑的函式
def load_path_graph(path_nodes, graph_file="graph.pkl"):
    import pickle
    with open(graph_file, "rb") as f:
        graph_data = pickle.load(f)

    # 建立 (u, v) ➜ geometry 查表，並強制轉成 (lat, lon)
    edge_lookup = {}
    for edge in graph_data:
        u, v = edge["u"], edge["v"]
        edge_lookup[(u, v)] = [(lat, lon) for lon, lat in edge["geometry"]]

    lines = []
    missing_edges = []
    for i in range(len(path_nodes) - 1):
        u, v = path_nodes[i], path_nodes[i + 1]
        if (u, v) in edge_lookup:
            lines.append(edge_lookup[(u, v)])
        elif (v, u) in edge_lookup:
            lines.append(list(reversed(edge_lookup[(v, u)])))
        else:
            print(f"⚠️ 找不到邊 {u} ➝ {v}，跳過")
            missing_edges.append((u, v))
            lines.append([])  # 保持長度一致
    return lines, missing_edges

# 畫路徑，只畫有資料的線段，並統計缺失
lines, missing_edges = load_path_graph(ucs_path)
missing_count = 0
for line in lines:
    if line:
        fmap.add_child(folium.PolyLine(locations=line, tooltip='ucs', weight=4, color='violet'))
    else:
        missing_count += 1
print(f"缺少 geometry 的線段數量：{missing_count}")
if missing_edges:
    print("缺失的邊：", missing_edges)

# 顯示地圖
fmap

The number of nodes in the path found by UCS: 102
Total distance of path found by UCS: 4313.8043736984255 m
The number of visited nodes in UCS: 16821

缺少 geometry 的線段數量：0
